# AMR SKID-STEERING PROJECT
### Notebook contents
* RRT* + steer implementation (motion planning)
* RRT* implementation (only path planning)
* Controller of skid-steering Robot, useful for the steering computation
### Notebook contributors
* Mattia Castelmare
* Andrea Giuseppe Di Francesco
* Enrico Fazzi

In [1]:
import numpy as np
import math
from env import *
from plotting import *
import copy
from utils import *
import matplotlib.pyplot as plt


## RRT* + steer

In [3]:
class Node:
    def __init__(self, x, y):

        self.x = x
        self.xd = None
        self.xdd = None

        self.y = y
        self.yd = None
        self.ydd = None

        self.theta = None
        self.thetad = None
        self.thetadd = None

        self.parent = None
        self.cost = None

        self.commands = None
        self.conf = None


class RRT_STAR:
    def __init__(self, start, goal, env, step, prob_gs, n_iters):

        self.node_s = Node(start[0], start[1])
        self.node_g = Node(goal[0], goal[1])

        ##################### INITIAL CONDITION #################
        self.node_s.theta = theta
        self.node_s.xd = xd
        self.node_s.xdd = xdd
        self.node_s.yd = yd
        self.node_s.ydd = ydd
        self.node_s.thetad = thetad
        self.node_s.thetadd = thetadd
        self.node_s.conf = [(self.node_s.x, self.node_s.y)]
        self.node_g.conf = [(self.node_g.x, self.node_g.y)]

        #########################################################

        self.env = env
        self.n_iters = n_iters
        self.step = step
        self.prob_gs = prob_gs  # Empirical value
        self.utils = Utils()
        self.node_s.cost = 0
        self.Tree = [self.node_s]
        self.plotting = Plotting(self.node_s, self.node_g)
        
        # Empirical values
        self.radius = max(self.env.x_range[-1], self.env.y_range[-1])//20
        self.trav_map = np.clip(np.random.normal(3, 8, size=(50, 30)), 2, 100)

    def planning(self):
        ''' This function computes the RRT* motion planning, thanks to the utilies and the Controller class.;
            args: ............................................................................................
            output: 
        '''
        for iter in range(self.n_iters):
            if iter % 100 == 0:
                print("ITERATION OF PLANNING N°: ",iter)
  
            n_rand = self.sample()
            n_nearest, n_new = self.compute_NN(n_rand)

            if n_nearest.x == n_new.x and n_nearest.y == n_new.y:
                continue
            
            ctr = Controller((n_nearest.x, n_nearest.y), (n_new.x, n_new.y))
            out_steer = ctr.steer(n_nearest.xd, n_nearest.xdd, n_nearest.yd, n_nearest.ydd, n_nearest.theta, n_nearest.thetad, n_nearest.thetadd)
            
            if out_steer != False:
                n_new_traj, xd, xdd, yd, ydd, thetaf, thetad, thetadd = out_steer
            else:
                continue

            n_new = Node(n_new_traj[-1][0], n_new_traj[-1][1])

            if not self.utils.is_collision(n_nearest, n_new) and int(n_new.x) in range(0, self.env.x_range[-1]) and int(n_new.y) in range(0, self.env.y_range[-1]):
                
                # Assign to node velocity, acceleration and commands
                n_new.xd = xd
                n_new.xdd = xdd
                n_new.yd = yd
                n_new.ydd = ydd
                n_new.theta = thetaf
                n_new.thetad = thetad
                n_new.thetadd = thetadd
                # n_new.commands = 0 #commands
                n_new.conf = n_new_traj

                N_near = self.compute_NN(n_new, near=True)
                n_min, cost_min = self.choose_parent(N_near, n_nearest, n_new)
                n_new.parent = n_min
                n_new.cost = cost_min
                # Insert Node
                self.Tree.append(n_new)
                self.rewire(N_near, n_min, n_new)

        path = [[self.node_g]]
        goal_state = self.get_goal_state()

        self.tot_path = self.compute_path(path, goal_state)

        self.plotting.animation(self.Tree, self.tot_path,
                                "rrt*, N = " + str(self.n_iters))

    def choose_parent(self, N_near, n_nearest, n_new):
        ''' This function change the parent of n_new, starting with an initial guess on its nearest neighbor.;
            args: (N_near : List of n_new's neighbors), (n_nearest : n_new's initial guess as a parent.),
                  (n_new : Node object that need to be assigned at a node in N_near).
            output: 
        '''
        n_parent = n_nearest
        # Init. guess parent for sampled node
        cost_min = n_nearest.cost + self.compute_cost(n_new.conf)

        for n_near in N_near:

            if n_near.x == n_new.x and n_near.y == n_new.y:
                continue

            ctr = Controller((n_near.x, n_near.y), (n_new.x, n_new.y))
            out_steer = ctr.steer(n_near.xd, n_near.xdd, n_near.yd, n_near.ydd, n_near.theta, n_near.thetad, n_near.thetadd)
            
            if out_steer != False:
                 n_new_traj, xd_, xdd_, yd_, ydd_, theta_, thetad_, thetadd_ = out_steer
            else:
                continue
            # print('after_steer')
            
            n_new_tr = Node(n_new_traj[-1][0], n_new_traj[-1][1])

            if not self.utils.is_collision(n_near, n_new_tr) and int(n_new_tr.x) in range(0, self.env.x_range[-1]) and int(n_new_tr.y) in range(0, self.env.y_range[-1]):

                inter_cost = n_near.cost + \
                    self.compute_cost(n_new_traj)  # Intermediate cost
                if n_new_tr.cost != None:
                    if inter_cost < n_new.cost and inter_cost < cost_min:
                        n_parent = n_near         # New best alternative
                        cost_min = inter_cost

                        n_new.xd = xd_
                        n_new.xdd = xdd_
                        n_new.yd = yd_
                        n_new.ydd = ydd_
                        n_new.theta = theta_
                        n_new.thetad = thetad_
                        n_new.thetadd = thetadd_
                        # n_new.commands = commands
                        n_new.conf = n_new_traj

                        n_new.x, n_new.y = n_new_tr.x, n_new_tr.y

                else:
                    if inter_cost < cost_min:
                        n_parent = n_near         # New best alternative
                        cost_min = inter_cost

                        n_new.xd = xd_
                        n_new.xdd = xdd_
                        n_new.yd = yd_
                        n_new.ydd = ydd_
                        n_new.theta = theta_
                        n_new.thetad = thetad_
                        n_new.thetadd = thetadd_
                        # n_new.commands = commands
                        n_new.conf = n_new_traj

                        n_new.x, n_new.y = n_new_tr.x, n_new_tr.y

        return n_parent, cost_min

    def rewire(self, N_near, n_parent, n_new):
        ''' This function change the parent of the n_new's neighbors iff n_new is a better parent than their current ones. ;
            args: (N_near : List of n_new's neighbors), (n_parent : n_new's parent is not considered in the computation.),
                  (n_new : Candidate parent, for the nodes in N_near).
            output: ........................................................................................................
        '''

        for n_near in N_near:
            if n_near.x == n_parent.x and n_near.y == n_parent.y:
                continue

            if n_near.x == n_new.x and n_near.y == n_new.y:
                continue
            
            ctr = Controller((n_new.x, n_new.y), (n_near.x, n_near.y))
            out_steer = ctr.steer(n_new.xd, n_new.xdd, n_new.yd, n_new.ydd, n_new.theta, n_new.thetad, n_new.thetadd)
          
            if out_steer != False:
                 n_new_traj, xd_, xdd_, yd_, ydd_, theta_, thetad_, thetadd_ = out_steer
            else:
                continue
            z_near = Node(n_new_traj[-1][0], n_new_traj[-1][1])

            if not self.utils.is_collision(z_near, n_new) and int(z_near.x) in range(0, self.env.x_range[-1]) and int(z_near.y) in range(0, self.env.y_range[-1]):

                if n_new.cost + self.compute_cost(n_new_traj) < n_near.cost:

                    z_near.parent = n_new
                    z_near.cost = n_new.cost + self.compute_cost(n_new_traj)

                    # n_near_old_idx = self.Tree.index(n_near)
                    # self.Tree[n_near_old_idx] = z_near

                    z_near.xd = xd_
                    z_near.xdd = xdd_
                    z_near.yd = yd_
                    z_near.ydd = ydd_
                    z_near.theta = theta_
                    z_near.thetad = thetad_
                    z_near.thetadd = thetadd_
                    # z_near.commands = commands
                    z_near.conf = n_new_traj

                    self.Tree.append(z_near)

        return

    def compute_NN(self, n_curr, near=False):
        ''' This function has two main objectives, either to compute a nearest neighbor, and also to compute a set of nearest neighbors. ;
            args: (n_curr : Node object, current node), (near : If True, are computed the neighbors for n_curr given a certain radius.)
            output: n_nearest, n_new XOR node_list
        '''
        # CHECK COLLISION
        dist_list = np.array([self.compute_dist_NN(n_curr, n_tree)
                             for n_tree in self.Tree])

        if not near:  # Nearest

            n_nearest = self.Tree[np.argmin(dist_list)]
            dist = min(np.min(dist_list), self.step)
            theta = math.atan2(n_curr.y-n_nearest.y, n_curr.x - n_nearest.x)
            n_new = Node(n_nearest.x + dist*math.cos(theta),
                         n_nearest.y + dist*math.sin(theta))

            return n_nearest, n_new
        else:        # Near
            node_list = [self.Tree[idx] for idx in range(
                len(dist_list)) if dist_list[idx] < self.radius]

            return node_list

    def sample(self):
        ''' Sample function RRT*, sample the goal node with a low probability '''
        if np.random.random() > self.prob_gs:
            x_new = np.random.uniform(self.env.x_range[-1])
            y_new = np.random.uniform(self.env.y_range[-1])
            return Node(x_new, y_new)

        return self.node_g

    def get_goal_state(self):
        ''' This function the closest state to the goal state, that was found during the algorithm;
            args: ........................................................................
            output: goal_state: Node object
        '''
        dist_list = np.array([self.compute_dist_NN(node, self.node_g)
                             for node in self.Tree])
        min_idx = np.argmin(dist_list)
        goal_state = self.Tree[min_idx]

        return goal_state

    def compute_path(self, path, goal_state):
        ''' This function fills the path between the start and goal state, it considers that each node has a parent ;
            args: (path : empty path), (goal_state : pre-computed goal node, which is the closest to the actual goal node.)
            output: path : complete path e.g. [[Node_g], [Node_g-1], ......, [Node_start]]
        '''
        n_curr = goal_state
        while n_curr.x != self.node_s.x and n_curr.y != self.node_s.y:
            path.append([n_curr])
            n_curr = n_curr.parent
        path.append([n_curr])

        return path

    def compute_dist_NN(self, n1, n2):
        ''' This function computes the euclidean distance between two object of the Node class 
            args: n1, n2 : n_i is an instance of Node ;
            
            output: dist : Total distance (m)

        '''
        dist = math.sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2)
        return dist

    def compute_dist(self, conf):
        ''' This function takes a trajectory of positions, and computes the distance (m) the links these coordinates 
            args: conf : [(x_0, y_0), (x_1, y_1), ......, (x_f, y_f)];
            
            output: dist : Total distance (m)

        '''
        dist = 0

        dist_func = lambda p1, p2: math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

        for pt in range(1, len(conf)):
            dist += dist_func(conf[pt], conf[pt - 1])
        return dist

    def compute_cost(self, conf):
        ''' This function takes a trajectory of positions, and computes the cost to do such trajectory, it is also used an elevation map. 
            args: conf : [(x_0, y_0), (x_1, y_1), ......, (x_f, y_f)];
            
            output: cost : Total distance (m)

        '''
        # Elevation on the real map
        n1_trav = self.trav_map[int(conf[-1][0]), int(conf[-1][1])]
        n2_trav = self.trav_map[int(conf[0][0]), int(conf[0][1])]

        increm = n2_trav - n1_trav  # Elevation between the two points
        plan_dist = self.compute_dist(conf)  # Planar distance

        cost = math.sqrt(plan_dist**2 + increm**2)

        return cost

        
    

 

## RRT NO steer
* In this cell there is a RRT* version, which is only path planning devoted

In [ ]:
class RRT_STAR_path:
    def __init__(self, start, goal, env, step, prob_gs, n_iters):

        self.node_s = Node(start[0], start[1])
        self.node_g = Node(goal[0], goal[1])

        #########################################################

        self.env = env
        self.n_iters = n_iters
        # CHECK IF START NODE AND GOAL NODE IN THE ENV......
        self.step = step
        self.prob_gs = prob_gs  # Empirical value
        self.utils = Utils()
        self.node_s.cost = 0
        self.Tree = [self.node_s]
        self.plotting = Plotting(self.node_s, self.node_g)
        # Empirical value
        self.radius = max(self.env.x_range[-1], self.env.y_range[-1])//15
        self.trav_map = np.clip(np.random.normal(3, 8, size=(50, 30)), 2, 100)

    def planning(self):
        # ax = sns.heatmap(self.trav_map.T, linewidth=0.5)
        # plt.show()
        for iter in range(self.n_iters):
            if iter % 100 == 0:
                print("ITERATION OF PLANNING N°: ",iter)
          
            n_rand = self.sample()
            n_nearest, n_new = self.compute_NN(n_rand)

            if n_nearest.x == n_new.x and n_nearest.y == n_new.y:
                continue

            if not self.utils.is_collision(n_nearest, n_new) and int(n_new.x) in range(0, self.env.x_range[-1]) and int(n_new.y) in range(0, self.env.y_range[-1]):
                # Assign to node velocity, acceleration and commands

                N_near = self.compute_NN(n_new, near=True)
                n_min, cost_min = self.choose_parent(N_near, n_nearest, n_new)
                n_new.parent = n_min
                n_new.cost = cost_min
                # Insert Node
                self.Tree.append(n_new)
                self.rewire(N_near, n_min, n_new)

        path = [[self.node_g]]
        goal_state = self.get_goal_state()
        print('ERROR IS: ', self.compute_dist_NN(self.node_g, goal_state))

        self.tot_path = self.compute_path(path, goal_state)

        self.plotting.animation(self.Tree, self.tot_path,
                                "rrt*, N = " + str(self.n_iters), steer=False)
    def choose_parent(self, N_near, n_nearest, n_new):
        n_parent = n_nearest
        # Init. guess parent for sampled node
        cost_min = n_nearest.cost + self.compute_cost(n_nearest, n_new)

        for n_near in N_near:
            if n_near.x == n_new.x and n_near.y == n_new.y:
                continue

            if not self.utils.is_collision(n_near, n_new) and int(n_new.x) in range(0, self.env.x_range[-1]) and int(n_new.y) in range(0, self.env.y_range[-1]):

                inter_cost = n_near.cost + \
                    self.compute_cost(n_near, n_new)  # Intermediate cost
                if n_new.cost != None:
                    if inter_cost < n_new.cost and inter_cost < cost_min:
                        n_parent = n_near         # New best alternative
                        cost_min = inter_cost


                else:
                    if inter_cost < cost_min:
                        n_parent = n_near         # New best alternative
                        cost_min = inter_cost

        return n_parent, cost_min

    def rewire(self, N_near, n_parent, n_new):
        for n_near in N_near:
            if n_near.x == n_parent.x and n_near.y == n_parent.y:
                continue

            if n_near.x == n_new.x and n_near.y == n_new.y:
                continue


            if not self.utils.is_collision(n_near, n_new) and int(n_near.x) in range(0, self.env.x_range[-1]) and int(n_near.y) in range(0, self.env.y_range[-1]):

                if n_new.cost + self.compute_cost(n_near, n_new) < n_near.cost:

                    n_near_old_idx = self.Tree.index(n_near)
                    n_near.parent = n_new
                    n_near.cost = n_new.cost + self.compute_cost(n_near, n_new)
                    self.Tree[n_near_old_idx] = n_near

        return

    def compute_NN(self, n_curr, near=False):

        # CHECK COLLISION
        dist_list = np.array([self.compute_dist(n_curr, n_tree)
                             for n_tree in self.Tree])

        if not near:  # Nearest

            n_nearest = self.Tree[np.argmin(dist_list)]
            dist = min(np.min(dist_list), self.step)
            theta = math.atan2(n_curr.y-n_nearest.y, n_curr.x - n_nearest.x)
            n_new = Node(n_nearest.x + dist*math.cos(theta),
                         n_nearest.y + dist*math.sin(theta))

            return n_nearest, n_new
        else:        # Near
            node_list = [self.Tree[idx] for idx in range(
                len(dist_list)) if dist_list[idx] < self.radius]

            return node_list

    def sample(self):
        ''' Sample function RRT*, sample the goal node with a low probability '''
        if np.random.random() > self.prob_gs:
            x_new = np.random.uniform(self.env.x_range[-1])
            y_new = np.random.uniform(self.env.y_range[-1])
            return Node(x_new, y_new)

        return self.node_g

    def get_goal_state(self):

        dist_list = np.array([self.compute_dist(node, self.node_g)
                             for node in self.Tree])
        min_idx = np.argmin(dist_list)
        goal_state = self.Tree[min_idx]

        return goal_state

    def compute_path(self, path, goal_state):

        n_curr = goal_state
        while n_curr.x != self.node_s.x and n_curr.y != self.node_s.y:
            path.append([n_curr])
            n_curr = n_curr.parent
        path.append([n_curr])

        return path

    def compute_dist(self, n1, n2):
        dist = math.sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2)
        return dist

    def compute_cost(self, n1, n2):
        n1_trav = self.trav_map[int(n1.x), int(n1.y)]
        n2_trav = self.trav_map[int(n2.x), int(n2.y)]

        increm = n2_trav - n1_trav  # Elevation between the two points
        plan_dist = self.compute_dist(n1, n2)  # Planar distance

        cost = math.sqrt(plan_dist**2 + increm**2)

        return cost

## Controller

In [ ]:
class Controller:

    def __init__(self, start, goal):
        
        ### Desired time to execute the trajectory ###
        self.T = math.sqrt((start[0]-goal[0])**2 + (start[1]-goal[1])**2)/(max_vel) + 1

        self.steps = np.arange(0, self.T+1/freq, 1/freq) # Discretized time steps, e.g. freq = 100, --> 100 Hz
        ### Initial and final points in the trajectory ###
        self.start = start
        self.goal = goal
        ### Collision check utils ###
        self.utils = Utils()
        
        ### Initialization of the trajectories ###
        self.timing_law(self.start, self.goal)
        

        ### Desired values on position, velocity, acceleration and jerk ###
        self.x = self.pos1(self.steps)
        self.y = self.pos2(self.steps)

        self.xd = self.vel1(self.steps)*np.ones(self.x.shape)
        self.yd = self.vel2(self.steps)

        self.xdd = self.acc1(self.steps)*np.ones(self.x.shape)
        self.ydd = self.acc2(self.steps)

        self.xddd = self.jerk1(self.steps)*np.ones(self.x.shape)
        self.yddd = self.jerk2(self.steps)*np.ones(self.y.shape)


    def timing_law(self, st, gl):
        ''' This function initialize the trajectory to be executed
            args: st, gl : tuple-like (x_0, y_0) 
            
            output: lambda functions, that can be fed with discretized time steps. '''
        
        self.pos1 = lambda t : st[0] + (t/self.T)*(gl[0]-st[0])
        self.pos2 = lambda t : st[1] + (1.0098*(t/self.T) - 0.0002*(t/self.T)**3 -0.0096*(t/self.T)**2)*(gl[1]-st[1])
        
        self.vel1 = lambda t : 1*(gl[0]-st[0])/(self.T)
        self.vel2 = lambda t : (-0.0006*t**2 - 0.0192*t + 1.0098)*(gl[1]-st[1])
        
        self.acc1 = lambda t : 0
        self.acc2 = lambda t : (-0.0012*t - 0.0192)*(gl[1]-st[1])

        self.jerk1 = lambda t : 0
        self.jerk2 = lambda t : -0.0012*(gl[1]-st[1])

        

    def steer(self, xd, yd, xdd, ydd, theta, thetad, thetadd):
        ''' This function computes the steering between the inizialized self.start, and self.goal, a controller tries to 
            converge at the desired trajectories, precomputed in self.timing_law
            args: xd, yd, xdd, ydd, theta, thetad, thetadd : float, scalars 
            
            output: False: If a collision is detected, otherwise
                    conf, xd, xdd, yd, ydd, thetaf, thetad, thetadd: conf is the set of configurations, and the others are the final 
                    quantities that are recorded to reach the goal node, these are stored so as to guarantee continuity '''
        commands = []

        # Initial position corrensponds with the s
        x = self.start[0]
        y = self.start[1]

        # Pseudo-velocities initial conditions: These are defined in the Robot reference frame (RF)
        eta10 = xd
        eta20 = yd 
        
        # Robot acceleration (w.r.t. the Robot RF)
        u10 = xdd

        conf = [(x, y)] # Initial configuration

        for state_idx in range(self.x.shape[0]):

            # The variable with the UNDERSCORE is the real one executed by the robot

            # The point that we want to control is shifted w.r.t. the robot origin, we compute the actual state
            z1_, z2_, z1d_, z2d_, z1dd_, z2dd_ = self.compute_state(
                x, y, xd, yd, theta, xdd, ydd, thetad, thetadd)

            # The control input are computed via a feedforward + feedback term
            r1 = self.xddd[state_idx] + ka*(self.xdd[state_idx] - z1dd_) + kv * \
                (self.xd[state_idx] - z1d_) + kp*(self.x[state_idx] - z1_)
                
            r2 = self.yddd[state_idx] + ka*(self.ydd[state_idx] - z2dd_) + kv * \
                (self.yd[state_idx] - z2d_) + kp*(self.y[state_idx] - z2_)

            # u1 is computed through an integrator on (eps_d)
            eps_d = math.cos(theta)*r1 + math.sin(theta) * \
                r2 + (1/(d0**2))*eta10*(eta20**2)
            u1 = np.clip(eps_d*1/freq + u10, -cmd_bd, cmd_bd)
            
            # u2 is the other command input (acceleration input)
            u2 = np.clip((d0/eta10)*(math.sin(theta)*r1 -
                        math.cos(theta)*r2)-(2/eta10)*u1*eta20, -cmd_bd, cmd_bd)
            

            # STATE RECONSTRUCTION #
            xf = x + (1/freq)*(math.cos(theta)*eta10 - math.sin(theta)*eta20)

            yf = y + (1/freq)*(math.sin(theta)*eta10 + math.cos(theta)*eta20) 
            
            if self.utils.is_inside_obs_alternative((xf, yf)):
                return False # Check on the new state validity (Collision check)

            thetaf = theta - (1/(d0*freq))*eta20

            eta1 = np.clip(u1*1/freq + eta10, -max_vel, max_vel)

            eta2 = np.clip(u2*1/freq + eta20, -max_vel, max_vel)

            thetad = -(eta20/d0)
            thetadd = -(u2/d0)

            eta10 = eta1
            eta20 = eta2
            u10 = u1

            xd = eta1
            yd = eta2

            xdd = u1
            ydd = u2
            theta = thetaf
            
            x = xf
            y = yf

            conf.append((xf, yf))
        
        return conf, xd, xdd, yd, ydd, thetaf, thetad, thetadd


    def compute_state(self, x, y, xd, yd, theta, xdd, ydd, thetad, thetadd):
        ''' This function compute the robot state considerating the shifting of the point to control. 
            args x, y, xd, yd, theta, xdd, ydd, thetad, thetadd: Coordinates. Except (x, y), they are computed in the Robot RF, thus formulas
                                                                 are particularly computed.

            output: Current state values, computed with the shifting took into account.
            '''
        z1_ = x + d0 * math.cos(theta)
        z2_ = y + d0 * math.sin(theta)

        z1d_ = xd*math.cos(theta) - yd*math.sin(theta) - \
                d0*math.sin(theta)*thetad
        z2d_ = xd*math.sin(theta) + yd*math.cos(theta) + \
                d0*math.cos(theta)*thetad

        z1dd_ = (xdd - yd*thetad)*math.cos(theta) - (ydd + xd*thetad)*math.sin(theta) - \
            d0*math.cos(theta)*(thetad**2) - d0*math.sin(theta)*thetadd

        z2dd_ = (xdd - yd*thetad)*math.sin(theta) + (ydd + xd*thetad)*math.cos(theta) - \
            d0*math.sin(theta)*(thetad**2) + d0*math.cos(theta)*thetadd 
        
        # z1dd_ = xdd*math.cos(theta) - yd*thetad*math.sin(theta) - d0 * \
        #         math.cos(theta)*thetad**2 - d0*math.sin(theta)*thetadd

        # z2dd_ = ydd*math.sin(theta) + xd*thetad*math.cos(theta) - d0 * \
        #         math.sin(theta)*thetad**2 + d0*math.cos(theta)*thetadd
      
        return z1_, z2_, z1d_, z2d_, z1dd_, z2dd_

## SIMULATION

In [ ]:
# Environment initialization #
env = Env(x_bounds=x_bounds, y_bounds=y_bounds)


### MOTION PLANNING ###
rrt = RRT_STAR(start, goal, env, step, prob_gs, n_iters) 

### ONLY PATH PLANNING ###
# rrt = RRT_STAR_path(start, goal, env, step, prob_gs, n_iters) 


In [ ]:
rrt.planning()

In [ ]:
for k in range(1, len(rrt.tot_path)):
    print(rrt.tot_path[-k][0].conf[0])
    print(rrt.tot_path[-k][0].conf[-1])

# Old planner

In [4]:
class Controller:

    def __init__(self, path, T):
        
        path.reverse()

        x_coords = [node[0].x for node in path]
        y_coords = [node[0].y for node in path]
        self.n_nodes = len(path)
        discrete_steps = np.arange(0, len(path), 1)
        self.T = T #tot_time
        
        self.xd0 = xd
        self.yd0 = yd
        self.theta0 = theta
        self.thetad0 = thetad
        self.xdd0 = xdd 
        self.ydd0 = ydd  
        self.thetadd0 = thetadd  

        self.timing_law()

        spl_x = UnivariateSpline(discrete_steps, x_coords, k=5)
        spl_y = UnivariateSpline(discrete_steps, y_coords, k=5)

        self.steps = np.arange(0, self.T, 1/freq)

        self.x = spl_x(self.s(self.steps))
        self.y = spl_y(self.s(self.steps))

        self.xd = spl_x.derivative(n = 1)(self.s(self.steps))
        self.yd = spl_y.derivative(n = 1)(self.s(self.steps))
        
        # self.theta = np.arctan2(self.yd, self.xd)
     
        self.xdd = spl_x.derivative(n = 2)(self.s(self.steps))
        self.ydd = spl_y.derivative(n = 2)(self.s(self.steps))

        # self.thetad = (self.ydd*self.xd - self.xdd*self.yd) / (self.xd**2 + self.yd(self.s(self.steps))**2)


        self.xddd = spl_x.derivative(n = 3)(self.s(self.steps))
        self.yddd = spl_y.derivative(n = 3)(self.s(self.steps))
        
        
        self.ex_path = [path[0]]
    
   
    def timing_law(self):

        self.s = lambda t: (t/self.T)**3 * (self.n_nodes - 1)

    

    def steer(self):

        qd0 = np.array([[self.xd0*math.cos(self.theta0) - self.yd0*math.sin(self.theta0)],
                        [self.xd0*math.sin(self.theta0) + self.yd0*math.cos(self.theta0)],
                        [self.thetad0]])

        commands = []
        conf = []
        u10 = 0
        
        eta10 = 0
        eta20 = 0


        

        x = self.x[0]
        y = self.y[0]
        theta = self.theta0

        xd = self.xd0
        yd = self.yd0
        thetad = self.thetad0
        
        xdd = self.xdd0
        ydd = self.ydd0
        thetadd = self.thetadd0

        for state_idx in range(self.x.shape[0]):

            # The variable with the UNDERSCORE is the real one executed by the robot
            z1_, z2_, z1d_, z2d_, z1dd_, z2dd_ = self.compute_state(
                x, y, xd, yd, theta, xdd, ydd, thetad, thetadd)

            print('real acc: ', z1dd_)
            print('real pos: ', z1_)
            print('real vel: ', z1d_)

            print('des acc: ', self.xdd[state_idx])
            print('des pos: ', self.x[state_idx])
            print('des vel: ', self.xd[state_idx])
            print('des jerk: ', self.xddd[state_idx])



            r1 = self.xddd[state_idx] + ka*(self.xdd[state_idx] - z1dd_) + kv * \
                (self.xd[state_idx] - z1d_) + kp*(self.x[state_idx] - z1_)
            r2 = self.yddd[state_idx] + ka*(self.ydd[state_idx] - z2dd_) + kv * \
                (self.yd[state_idx] - z2d_) + kp*(self.y[state_idx] - z2_)


            print('R1: ', r1)
            print('R2: ', r2)


            Rx, Fy, Mr = self.compute_friction(xd, yd, thetad)

            # print("Frictions: ", Rx, Fy, Mr)
            
            eps_d = math.cos(theta)*r1 + math.sin(theta) * \
                r2 + 1/d0*xd*yd**2
            
            print('eps_d: ', eps_d)



            u1 = eps_d*1/freq + u10
            u10 = u1
            # print("u1: ",u1)
            print('u1/xdd: ', u1)

            u2 = (d0/xd)*(math.sin(theta)*r1 -
                        math.cos(theta)*r2)-(2/xd)*u1*yd
            print('u2/ydd: ', u2)
            
            
            # print("u2: ",u2)
            
            # tau1 = r/2*(mass * u1 + mass/d0 * (yd**2) +Rx) - ((r*d0) / (
            #     2*t)) * ((mass + (Iz/d0**2)) * u2 - (mass/d0)*xd*yd + Fy - Mr/d0)
            # tau2 = r/2*(mass * u1 + mass/d0 * (yd**2) +Rx) + ((r*d0) / (
            #     2*t)) * ((mass + (Iz/d0**2)) * u2 - (mass/d0)*xd*yd + Fy - Mr/d0)

            #print('ERROR mass: ', mass * u1)#
            #print('Error mass2: ', mass/d0 * (yd**2))
            #print('ERROR Rx : ', Rx)
            #print("\n")
            
            #print('ERROR mass2: ', (r*d0) / (2*t)) 
            #print('Error mass3: ',  (mass + (Iz/d0**2)) )
            #print('Error mass4: ', u2 - mass/d0*xd*yd)
            #print('Error mass3 * mass 4: ',  (mass + (Iz/d0**2))* u2 - mass/d0*xd*yd)
            
            #print('ERROR Fy: ', Fy)
            #print("\n")

            #print('ERROR Mr/d0: ', Mr/d0)
            #print("\n") 
            # print('ERROR tau1: ', tau1)
            # print('ERROR tau2: ', tau2)
            # print("\n")


            # if tau1 > tau_max or tau1 < - tau_max:
            #     tau1 = np.sign(tau1)*tau_max
            # if tau2 > tau_max or tau2 < - tau_max:
            #     tau2 = np.sign(tau2)*tau_max

            #print('ERROR tau1: ', tau1)
            #print('ERROR tau2: ', tau2)

            # v, w, qdf, xd, yd, xdd, ydd, thetad, thetadd = self.compute_control(
            #     tau1, tau2, Rx, Fy, Mr, theta, qd0)
            # print('Acceleration ACC X: ', xdd)
            # print('Acceleration ACC Y: ', ydd)

            eta1 = u1*1/freq + eta10
            eta2 = u2*1/freq + eta20

            print('eta1/xd: ', eta1)
            print('eta2/yd: ', eta2)



            eta10 = eta1
            eta20 = eta2

            xf = x + (1/freq)*(math.cos(theta)*eta1 - math.sin(theta)*eta2)

            yf = y + (1/freq)*(math.sin(theta)*eta1 - math.cos(theta)*eta2) 

            thetaf = theta - (1/(d0*freq))*eta2


            print('xf: ', xf)
            print('yf: ', yf)
            print('thetaf: ', thetaf)


            # qd0 = qdf

            # commands.append((v, w))
            
            # print(theta, w)
            # xf, yf, thetaf = self.RK_integration(x, y, theta, 1/freq, v, w)
            conf.append((xf, yf))

            xd = eta1
            yd = eta2
            thetad = -(yd/d0)

            print('thetad: ', thetad)

            xdd = u1
            ydd = u2
            thetadd = -(ydd/d0)

            # print('XD: ', xd)
            # print('YD: ', yd)
            # print('THETAD: ', thetad)
            # print('XDD: ', xdd)
            # print('YDD: ', ydd)
            # print('THETADD: ', thetadd)



            theta = thetaf
            x = xf
            y = yf


        return conf, commands#n_new, xd, xdd, yd, ydd, thetaf, thetad, thetadd, commands, conf

    def RK_integration(self, x, y, theta, T, v, w):
        ''' Runge Kutta integration for a generic differential drive robot'''
        xf = x+v*T*math.cos(theta*(w*T)/2)
        yf = y+v*T*math.sin(theta*(w*T)/2)
        thetaf = theta+w*T

        return xf, yf, thetaf

    def compute_friction(self, xd, yd, thetad):
        '''This function computes the resistive force on the x axis Rx, the viscous friction on the y axis Fy and the resistive moment Mr'''
        x1d = x4d = xd - t*thetad
        x3d = x2d = xd + t*thetad

        y1d = y2d = yd + a*thetad

        y3d = y4d = yd - b*thetad

        Rx = fr * (mass * g)/2 * (np.sign(x1d) + np.sign(x2d))

        Fy = mu * ((mass*g)/(a + b)) * \
            (b*np.sign(y1d) + a*np.sign(y3d))
        
        Mr = mu * ((a*b*mass*g)/(a + b))*(np.sign(y1d) -
                                          np.sign(y3d)) + fr*t*mass*g/2*(np.sign(x2d) - np.sign(x1d))
        return Rx, Fy, Mr

    def compute_control(self, tau1, tau2, Rx, Fy, Mr, theta, qd0):
        '''This function computes the commands v and w useful for the Runge Kutta integration '''
        M = np.array([[mass, 0, 0], [0, mass, 0], [0, 0, Iz]])

        c = np.array([[Rx*math.cos(theta) - Fy*math.sin(theta)],
                     [Rx*math.sin(theta) + Fy*math.cos(theta)],
                      [Mr]])
        E = np.array([[math.cos(theta)/r, math.cos(theta)/r],
                      [math.sin(theta)/r, math.sin(theta)/r],
                      [t/r, -t/r]])
        tau = np.array([[tau1], [tau2]])

        R = np.array([[math.cos(theta), -math.sin(theta), 0],
                      [math.sin(theta), math.cos(theta), 0],
                      [0, 0, 1]])

        # print('E: ', E)
        # print('tau: ', tau)
        # print('c: ', c)



        qdd = np.linalg.inv(M)@(E@tau - c)

        # print("acc: ", qdd)

        qd = qdd*1/freq + qd0

        # print("vel: ", qd)


        vel = R.T@qd
        xd = vel[0]
        yd = vel[1]

        acc = R.T@qdd
        xdd = acc[0] + vel[1]*vel[2]
        ydd = acc[1] - vel[0]*vel[2]
        thetadd = qdd[2, 0]
        thetad = qd[2, 0]
        # print("xd: ", qd[0])
        # print("xdd: ", qdd[0])

        # print("yd: ", qd[1])
        # print("ydd: ", qdd[1])

        v = math.sqrt(qd[0]**2 + qd[1]**2)
        w = (qdd[1]*qd[0] - qdd[0]*qd[1]) / (qd[0]**2 + qd[1]**2)

        return float(v), float(w[0]), float(qd[0]), float(xd[0]), float(yd[0]), float(xdd[0]), float(ydd[0]), float(thetad), float(thetadd)

    def compute_state(self, x, y, xd, yd, theta, xdd, ydd, thetad, thetadd):
        
        z1_ = x + d0 * math.cos(theta)
        z2_ = y + d0 * math.sin(theta)

        z1d_ = xd*math.cos(theta) - yd*math.sin(theta) - \
                d0*math.sin(theta)*thetad
        z2d_ = xd*math.sin(theta) + yd*math.cos(theta) + \
                d0*math.cos(theta)*thetad

        z1dd_ = xdd*math.cos(theta) - yd*thetad*math.sin(theta) - d0 * \
                math.cos(theta)*thetad**2 - d0*math.sin(theta)*thetadd

        z2dd_ = ydd*math.sin(theta) + xd*thetad*math.cos(theta) - d0 * \
                math.sin(theta)*thetad**2 + d0*math.cos(theta)*thetadd
      
        return z1_, z2_, z1d_, z2d_, z1dd_, z2dd_

In [ ]:
import numpy as np
map = np.load("numpy_maps/lunar_tranquillitatis_pit.npy")

def is_transition_feasible(state, prev_state):

      x, y = state
      
      x_old, y_old = prev_state

      res = 2
      tol = 0.8

      z = 1 #map[int(x*res), int(y*res)]
                        
      points = [[1, x, y]]   # -cz = ax + by + d (c = -1) ; [0 = ax + by +cz + d]
      output = [[z]]

      options = [(1, 1), (1, -1), (-1, -1), (-1, +1), (1, 0), (0, 1)] # Square edges

      for opt in options:
            x_new = tol*opt[0] + x
            y_new = tol*opt[1] + y
            z_new = 1 #map[int(x_new*res), int(y_new*res)]

            points.append([1, x_new, y_new])
            output.append([z_new])
      
      X = np.array(points)
      y = np.array(output)

      # LEAST SQUARES METHOD
      w = np.linalg.pinv(X) @ y

      n = np.array([w[1][0], w[2][0], -1])        
      norm = np.linalg.norm(n)
      
      w_n = n/norm # Cosine directors.
      print("NORMALE", w_n)
      nx = w_n[0]
      ny = w_n[1]
      nz = -w_n[2]

      
      yaw = math.atan2(ny,nx)
      pitch = math.atan2(nx,nz)
      roll = math.atan2(nz,ny)

      print("Yaw angle (theta_z): degrees", np.degrees(yaw))
      print("Pitch angle (theta_y): degrees", np.degrees(pitch))
      print("Roll angle (theta_x): degrees", np.degrees(roll))

      pitch_bound = math.pi/4 # 45 degree
      roll_bound = math.pi/6 # 30 degree

      if pitch_bound <= pitch <= 2*math.pi - pitch_bound:
            return True

      if roll <= - roll_bound + math.pi/2 or roll >= roll_bound + math.pi/2:
            return True

      return False, math.hypot(x-x_old, y-y_old)
        
        


is_transition_feasible((3, 2.2))